In [1]:
# import wandb
from mmcv import Config
from mmseg.datasets import build_dataset

from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.datasets import (build_dataloader, build_dataset)
import mmcv

In [2]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
import os.path as osp
import numpy as np
from PIL import Image
import os
from mmseg.apis import set_random_seed

data_root = '/opt/ml/input/last'
classes = ('Background','General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]

In [3]:
# model 이름
model_path = '/opt/ml/mmsegmentation/_MyModel/_models_'
model_name = 'knet_s3_upernet_convnext-d8_8x2_512x512.py'
cfg = Config.fromfile(os.path.join(model_path,model_name))

In [4]:
# cfg.norm_cfg = dict(type='BN', requires_grad=True)
# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # cfg.model.neck.norm_cfg = cfg.norm_cfg
# cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# # cfg.model.decode_head.num_classes = 11
# # cfg.model.auxiliary_head.num_classes = 11

# # data root
# cfg.data_root = data_root

# batch size
cfg.data.samples_per_gpu = 6
cfg.data.workers_per_gpu=8
cfg.load_from='/opt/ml/mmsegmentation/_MyModel/work_dirs/knet_last/epoch_40_only.pth'

# # cfg.data.train.data_root = cfg.data_root
# # cfg.data.train.img_dir = 'images'
# # cfg.data.train.ann_dir = 'annotations'
# # cfg.data.train.split = data_root+'/splits/train.txt'


# cfg.data.val.data_root = cfg.data_root
# cfg.data.val.img_dir = 'images'
# cfg.data.val.ann_dir = 'annotations'
# cfg.data.val.split = data_root+'/splits/valid.txt'


# # where checkpoints saved
cfg.work_dir = './work_dirs/knet_last'

cfg.runner.max_epochs = 10 # max epoch

cfg.log_config.interval = 200 # train 정보 출력 간격
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=6, interval=2) # max_keep_ckpts : pht 파일 최대 저장 갯수 , interval : pth 저장 간격
# cfg.optimizer_config= dict(type='Fp16OptimizerHook', loss_scale='dynamic',grad_clip = dict(max_norm=35, norm_type=2))
# # Set seed to facitate reproducing the result
cfg.seed = 21
# set_random_seed(0, deterministic=False)
cfg.gpu_ids = [0]


In [5]:
datasets = build_dataset(cfg.data.train)


2022-05-11 19:52:16,221 - mmseg - INFO - Loaded 4417 images


In [6]:
# import torch
# import matplotlib.pyplot as plt
# import cv2
# import numpy as np
# from PIL import Image
# ## 
# datasets = build_dataset(cfg.data.train)


# fig, ax = plt.subplots(1,2,figsize=(12,12))
# data=datasets[0]
# test_img=data['img'].data.permute([1,2,0])
# test_mask=list(data['gt_semantic_seg'].data.flatten())
# for k in range(len(test_mask)):
#     if test_mask[k]==255:
#         test_mask[k]=(0,0,0)
#     else:       
#         test_mask[k]=palette[int(test_mask[k])]

# test_mask=torch.Tensor(test_mask).reshape(512,512,3)
# ax[0].imshow(test_img)
# ax[1].imshow(test_mask/255)

# cv2.imwrite('./test_img.jpg',test_img.numpy())
# test_mask=Image.fromarray(test_mask.numpy().astype(np.uint8))
# test_mask.save('./test_mask.jpg')

In [6]:
model = build_segmentor(cfg.model)
model.init_weights()

In [7]:
train_segmentor(model, datasets, cfg, distributed=False, validate=False,meta=dict())
# 모델 학습


2022-05-11 20:03:23,986 - mmseg - INFO - Epoch [1][200/736]	lr: 1.291e-05, eta: 6:27:09, time: 3.244, data_time: 0.022, memory: 25631, decode.loss_ce.s0: 0.0766, decode.acc_seg.s0: 97.5755, decode.loss_ce.s1: 0.0706, decode.acc_seg.s1: 97.7252, decode.loss_ce.s2: 0.0702, decode.acc_seg.s2: 97.7312, decode.loss_ce.s3: 0.0704, decode.acc_seg.s3: 97.7191, aux.loss_ce: 0.0481, aux.acc_seg: 96.1683, loss: 0.3358, grad_norm: 3.7763
2022-05-11 20:14:08,664 - mmseg - INFO - Epoch [1][400/736]	lr: 2.516e-05, eta: 6:15:07, time: 3.223, data_time: 0.005, memory: 25631, decode.loss_ce.s0: 0.0779, decode.acc_seg.s0: 97.5005, decode.loss_ce.s1: 0.0701, decode.acc_seg.s1: 97.7114, decode.loss_ce.s2: 0.0695, decode.acc_seg.s2: 97.7370, decode.loss_ce.s3: 0.0696, decode.acc_seg.s3: 97.7378, aux.loss_ce: 0.0490, aux.acc_seg: 96.1425, loss: 0.3361, grad_norm: 3.6399
2022-05-11 20:24:52,790 - mmseg - INFO - Epoch [1][600/736]	lr: 3.669e-05, eta: 6:03:51, time: 3.221, data_time: 0.005, memory: 25631, decod